In [4]:
from core.model import llm0
from langchain.indexes import GraphIndexCreator
from langchain.chains import GraphQAChain
from langchain.prompts import PromptTemplate

text = "Where are you from?"

index_creator = GraphIndexCreator(llm=llm0)
graph = index_creator.from_text(text)

print(graph.get_triples())

[]
